# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [134]:
# Dependencies and Setup
import hvplot.pandas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import json
import warnings
warnings.filterwarnings("ignore")
from pprint import pprint 

# Import API key
from api_keys import geoapify_key

In [135]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
dropna_city_data = city_data_df.dropna()

# Display sample data
dropna_city_data

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (C),Wind Speed (mph)
0,0,puerto ayora,EC,2023-02-07 17:30:55,-0.7393,-90.3518,29.0,91.0,28.98,4.47
1,1,nikolskoye,RU,2023-02-07 17:33:41,59.7035,30.7861,99.0,88.0,-1.06,4.50
2,2,barrow,US,2023-02-07 17:33:41,71.2906,-156.7887,100.0,78.0,-18.99,0.00
3,3,punta arenas,CL,2023-02-07 17:30:40,-53.1500,-70.9167,75.0,59.0,16.06,6.69
4,4,kaseda,JP,2023-02-07 17:33:42,31.4167,130.3167,96.0,77.0,11.11,5.29
...,...,...,...,...,...,...,...,...,...,...
586,586,chapais,CA,2023-02-07 17:39:59,49.7834,-74.8492,100.0,95.0,-7.12,5.37
587,587,beaverlodge,CA,2023-02-07 17:40:00,55.2166,-119.4361,91.0,89.0,-0.36,1.10
588,588,cumberland,US,2023-02-07 17:40:00,35.0502,-78.8664,0.0,31.0,18.28,4.63
589,589,sable-sur-sarthe,FR,2023-02-07 17:40:00,47.8376,-0.3329,8.0,83.0,1.63,3.29


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [136]:
map_plot_1 = dropna_city_data.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 800,
    scale = 1,
    alpha = 0.5,
    size = "Humidity (%)",
    color = "City" 
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [137]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
no_cloud = dropna_city_data[dropna_city_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 21 degrees C
above_21_degrees = no_cloud[no_cloud["Max Temp (C)"] > 21]
# Filter vacation with max temp below 27 degrees C
below_27_degrees = above_21_degrees[above_21_degrees["Max Temp (C)"] < 27]
# Filter vacation with wind speed below 10 mph
ideal_vacation = below_27_degrees[below_27_degrees["Wind Speed (mph)"] < 4.5]
# Set Index
indexed_ideal_vacation = ideal_vacation.reset_index()
del indexed_ideal_vacation["index"]


# Display sample data
indexed_ideal_vacation

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (C),Wind Speed (mph)
0,184,beitbridge,ZW,2023-02-07 17:35:43,-22.2167,30.0000,0.0,66.0,24.02,3.84
1,274,comodoro rivadavia,AR,2023-02-07 17:35:14,-45.8667,-67.5000,0.0,23.0,24.92,4.12
2,307,tautira,PF,2023-02-07 17:36:46,-17.7333,-149.1500,0.0,65.0,26.53,2.29
3,343,salalah,OM,2023-02-07 17:37:36,17.0151,54.0924,0.0,43.0,21.05,1.54
4,358,podor,MR,2023-02-07 17:37:42,16.6527,-14.9587,0.0,43.0,22.97,1.03
5,369,koungou,YT,2023-02-07 17:37:46,-12.7336,45.2042,0.0,83.0,26.84,3.60
6,384,russell,US,2023-02-07 17:36:40,32.3502,-85.1999,0.0,40.0,21.60,2.06
7,422,panguipulli,CL,2023-02-07 17:38:08,-39.6333,-72.3333,0.0,36.0,24.92,2.55
8,428,abu dhabi,AE,2023-02-07 17:28:39,24.4667,54.3667,0.0,31.0,26.01,2.57
9,517,illapel,CL,2023-02-07 17:36:12,-31.6308,-71.1653,0.0,50.0,26.20,2.76


### Step 3: Create a new DataFrame called `hotel_df`.

In [138]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = indexed_ideal_vacation.loc[:, ["City", "Country", "Latitude",
                                    "Longitude", "Humidity (%)",]]
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

,City,Country,Latitude,Longitude,Humidity (%),Hotel Name
0,beitbridge,ZW,-22.2167,30.0000,66.0,
1,comodoro rivadavia,AR,-45.8667,-67.5000,23.0,
2,tautira,PF,-17.7333,-149.1500,65.0,
3,salalah,OM,17.0151,54.0924,43.0,
4,podor,MR,16.6527,-14.9587,43.0,
5,koungou,YT,-12.7336,45.2042,83.0,
6,russell,US,32.3502,-85.1999,40.0,
7,panguipulli,CL,-39.6333,-72.3333,36.0,
8,abu dhabi,AE,24.4667,54.3667,31.0,
9,illapel,CL,-31.6308,-71.1653,50.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

### I have used category as acccommodation.hotel and not accomodation as hotel specific category is present in the database. Also, using accomodation as a category is also leading to Motels being included in the results and the question has asked to include hotels.

In [139]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    filters = f"circle:{lng},{lat},{radius}"
    bias = f"proximity:{lng},{lat}"
    limit = 1
    params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    #name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["address_line1"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
beitbridge - nearest hotel: Beitbridge Inn & Casino
comodoro rivadavia - nearest hotel: Leandro Alem
tautira - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
podor - nearest hotel: La Cour du Fleuve
koungou - nearest hotel: Hôtel Trévani
russell - nearest hotel: No hotel found
panguipulli - nearest hotel: Cabañas Villa Los Encinos
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
illapel - nearest hotel: Domingo Ortiz de Rosas


,City,Country,Latitude,Longitude,Humidity (%),Hotel Name
0,beitbridge,ZW,-22.2167,30.0000,66.0,Beitbridge Inn & Casino
1,comodoro rivadavia,AR,-45.8667,-67.5000,23.0,Leandro Alem
2,tautira,PF,-17.7333,-149.1500,65.0,No hotel found
3,salalah,OM,17.0151,54.0924,43.0,Muscat International Hotel
4,podor,MR,16.6527,-14.9587,43.0,La Cour du Fleuve
5,koungou,YT,-12.7336,45.2042,83.0,Hôtel Trévani
6,russell,US,32.3502,-85.1999,40.0,No hotel found
7,panguipulli,CL,-39.6333,-72.3333,36.0,Cabañas Villa Los Encinos
8,abu dhabi,AE,24.4667,54.3667,31.0,Marriott Downtown Abu-Dhabi
9,illapel,CL,-31.6308,-71.1653,50.0,Domingo Ortiz de Rosas


In [140]:
#Deleteing rows without hotels in 10000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df.reset_index(drop = True)

,City,Country,Latitude,Longitude,Humidity (%),Hotel Name
0,beitbridge,ZW,-22.2167,30.0000,66.0,Beitbridge Inn & Casino
1,comodoro rivadavia,AR,-45.8667,-67.5000,23.0,Leandro Alem
2,salalah,OM,17.0151,54.0924,43.0,Muscat International Hotel
3,podor,MR,16.6527,-14.9587,43.0,La Cour du Fleuve
4,koungou,YT,-12.7336,45.2042,83.0,Hôtel Trévani
5,panguipulli,CL,-39.6333,-72.3333,36.0,Cabañas Villa Los Encinos
6,abu dhabi,AE,24.4667,54.3667,31.0,Marriott Downtown Abu-Dhabi
7,illapel,CL,-31.6308,-71.1653,50.0,Domingo Ortiz de Rosas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [145]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    alpha = 1,
    size = "Humidity (%)",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%),Hotel Name,Country)